In [1]:
import pandas as pd

## Load data

In [2]:
stock_Brazil=pd.read_csv("Stock major indicies/Bovespa Historical Data_Brazil.csv")
stock_Brazil['Country']='Brazil'

stock_India=pd.read_csv("Stock major indicies/BSE Sensex 30 Historical Data_India.csv")
stock_India['Country']='India'

stock_France=pd.read_csv("Stock major indicies/CAC 40 Historical Data_Paris.csv")
stock_France['Country']='France'

stock_Germany=pd.read_csv("Stock major indicies/DAX Historical Data_Germany.csv")
stock_Germany['Country']='Germany'

stock_US=pd.read_csv("Stock major indicies/Dow Jones Industrial Average Historical Data_US.csv")
stock_US['Country']='US'

stock_UK=pd.read_csv("Stock major indicies/FTSE 100 Historical Data_UK.csv")
stock_UK['Country']='UK'

stock_Italy=pd.read_csv("Stock major indicies/FTSE MIB Historical Data_Italy.csv")
stock_Italy['Country']='Italy'

stock_Spain=pd.read_csv("Stock major indicies/IBEX 35 Historical Data_Spain.csv")
stock_Spain['Country']='Spain'

stock_Korea=pd.read_csv("Stock major indicies/KOSPI Historical Data_South Korea.csv")
stock_Korea['Country']='Korea'

stock_Japan=pd.read_csv("Stock major indicies/Nikkei 225 Historical Data_Japan.csv")
stock_Japan['Country']='Japan'

stock_Denmark=pd.read_csv("Stock major indicies/OMX Copenhagen 20 Historical Data_Denmark.csv")
stock_Denmark['Country']='Denmark'

stock_Australia=pd.read_csv("Stock major indicies/S&P_ASX 200 Historical Data_Austrila.csv")
stock_Australia['Country']='Australia'

stock_Canada=pd.read_csv("Stock major indicies/S&P_TSX Composite Historical Data_Canada.csv")
stock_Canada['Country']='Canada'

stock_China=pd.read_csv("Stock major indicies/Shanghai Composite Historical Data_China.csv")
stock_China['Country']='China'

stock_SouthAfrica=pd.read_csv("Stock major indicies/South-Africa-Top-40-Historical-Data_South-Africa.csv")
stock_SouthAfrica['Country']='South Africa'

In [3]:
stock_data=pd.concat([stock_Brazil,stock_India,stock_France,stock_Germany,stock_US,stock_UK,stock_Italy,stock_Spain,stock_Korea,stock_Japan,stock_Denmark,stock_Australia,stock_Canada,stock_China,stock_SouthAfrica])

# Data preparation

We are gonna focus on data from October 2019 to 14th of Auguts 2020

In [4]:
stock_data['Date']=pd.to_datetime(stock_data['Date'])
stock_data=stock_data.loc[(stock_data['Date'] > '30-09-2019')]

The remaining columns are changed to float

In [5]:
stock_data['Price']=stock_data.Price.str.replace(",", "").apply(lambda l: float(l))
stock_data['Open']=stock_data.Open.str.replace(",", "").apply(lambda l: float(l))
stock_data['High']=stock_data.High.str.replace(",", "").apply(lambda l: float(l))
stock_data['Low']=stock_data.Low.str.replace(",", "").apply(lambda l: float(l))
stock_data['Change %']=stock_data['Change %'].str.replace('%','').apply(lambda l:float(l))
stock_data=stock_data.rename(columns={"Change %": "Change"})

The Vol. column is dropped as it does not represent a value of intrest for this project.

In [6]:
stock_data=stock_data.drop(['Vol.'],axis=1)

## Line plot

https://realpython.com/python-data-visualization-bokeh/
https://docs.bokeh.org/en/latest/docs/user_guide/styling.html

In [7]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource,FactorRange
from bokeh.plotting import figure
from bokeh.palettes import viridis
from bokeh.models import Legend
from bokeh.models import HoverTool
from collections import OrderedDict
output_notebook()

Loading BokehJS ...

In [8]:
fig = figure(title='Daily change in stock marked',title_location='above',
            plot_width=1000,plot_height=500,
            y_axis_label="% daily change",
            x_axis_label="Date", x_axis_type="datetime",
            tools="reset,save")

In [9]:
countries=list(stock_data.Country.unique())
colors=viridis(len(stock_data.Country.unique()))
line={}
items=[]
for i in range(len(countries)):
     source = ColumnDataSource(
     data={'Date':stock_data.loc[stock_data.Country== countries[i]].Date,
           'Country':stock_data.loc[stock_data.Country == countries[i]].Country,
           'Change':stock_data.loc[stock_data.Country == countries[i]].Change})
     line[countries[i]]=fig.line(x='Date',y='Change', source=source, color = (colors[i]))
     items.append((countries[i], [line[countries[i]]]))

In [10]:
legend = Legend(items=items, location=(0, 0))
hover = HoverTool(tooltips =[
     ('Country','@Country'),('Date','@Date{%F}'),('Change','@Change')],formatters={'@Date': 'datetime'})
fig.add_tools(hover)
fig.add_layout(legend, 'right')
fig.legend.click_policy="hide"
show(fig)

It is clear that something drastic starts happening in the stock markeds around the world around the end of feburary, which intensify in the begining of Marts. This coorsponds with the time around which the first cases starts being confirmed outside of china. However it is difficult to really get an overview of the of the differences between the differen countries. The only really distinct thing which can be observed is that China experience a drastic drop already in the end of January. 

### Price

In [14]:
fig = figure(title='Daily stock price',title_location='above',
            plot_width=1000,plot_height=500,
            y_axis_label="Price in national valuta",
            x_axis_label="Date", x_axis_type="datetime",
            tools="reset,save")

In [15]:
line={}
items=[]
for i in range(len(countries)):
     source = ColumnDataSource(
     data={'Date':stock_data.loc[stock_data.Country== countries[i]].Date,
           'Country':stock_data.loc[stock_data.Country == countries[i]].Country,
           'Price':stock_data.loc[stock_data.Country == countries[i]].Price})
     line[countries[i]]=fig.line(x='Date',y='Price', source=source, color = (colors[i]))
     items.append((countries[i], [line[countries[i]]]))

In [16]:
legend = Legend(items=items, location=(0, 0))
hover = HoverTool(tooltips =[
     ('Country','@Country'),('Date','@Date{%F}'),('Price','@Price')],formatters={'@Date': 'datetime'})
fig.add_tools(hover)
fig.add_layout(legend, 'right')
fig.legend.click_policy="hide"
show(fig)

When focusing on the price it is easier to see that all the stock markeds have experienced a change following approximatly the same pattern since around the end of Febuary. However the plot is also missleading in the sense that the prices are not all in the same valuta, and comparing the numerical price in Brazill with that in germany does not make sense.

What both plots however make clear is that all the stock markeds have been affect by COVID-19. Indicating the financial consequences also introduced by the virus.